Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/automated-machine-learning/regression-car-price-model-explaination-and-featurization/auto-ml-regression.png)

# Automated Machine Learning
_**Regression with Aml Compute**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Results](#Results)
1. [Test](#Test)


## Introduction
In this example we use the Hardware Performance Dataset to showcase how you can use AutoML for a simple regression problem. The Regression goal is to predict the performance of certain combinations of hardware parts.
After training AutoML models for this regression data set, we show how you can compute model explanations on your remote compute using a sample explainer script.

If you are using an Azure Machine Learning Notebook VM, you are all set.  Otherwise, go through the [configuration](../../../configuration.ipynb)  notebook first if you haven't already to establish your connection to the AzureML Workspace. 

An Enterprise workspace is required for this notebook. To learn more about creating an Enterprise workspace or upgrading to an Enterprise workspace from the Azure portal, please visit our [Workspace page.](https://docs.microsoft.com/azure/machine-learning/service/concept-workspace#upgrade) 

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Instantiating AutoMLConfig with FeaturizationConfig for customization
3. Train the model using remote compute.
4. Explore the results and featurization transparency options
5. Setup remote compute for computing the model explanations for a given AutoML model.
6. Start an AzureML experiment on your remote compute to compute explanations for an AutoML model.
7. Download the feature importance for engineered features and visualize the explanations for engineered features. 
8. Download the feature importance for raw features and visualize the explanations for raw features. 


## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
import azureml.dataprep as dprep
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

In [ ]:
ws = Workspace.from_config()

# Choose a name for the experiment.
experiment_name = 'automl-regression-hardware-explain'
experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster-5"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 4)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

### Setup Training and Test Data for AutoML experiment

Load the hardware dataset from a csv file containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. Next, we'll split the data using random_split and extract the training data for the model.  We also register the datasets in your workspace using a name so that these datasets may be accessed from the remote compute.

In [ ]:
data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/machineData.csv'

dataset = Dataset.Tabular.from_delimited_files(data)

# Split the dataset into train and test datasets
train_data, test_data = dataset.random_split(percentage=0.8, seed=223)


# Register the train dataset with your workspace
train_data.register(workspace = ws, name = 'machineData_train_dataset',
                       description = 'hardware performance training data',
                      create_new_version=True)

# Register the test dataset with your workspace
test_data.register(workspace = ws, name = 'machineData_test_dataset', description = 'hardware performance test data', create_new_version=True)

label ="ERP"

train_data.to_pandas_dataframe().head()

## Train

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification, regression or forecasting|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**experiment_timeout_hours**| Maximum amount of time in hours that all iterations combined can take before the experiment terminates.|
|**enable_early_stopping**| Flag to enble early termination if the score is not improving in the short term.|
|**featurization**| 'auto' / 'off' / FeaturizationConfig Indicator for whether featurization step should be done automatically or not, or whether customized featurization should be used. Note: If the input data is sparse, featurization cannot be turned on.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|(sparse) array-like, shape = [n_samples, n_features]|
|**label_column_name**|(sparse) array-like, shape = [n_samples, ], targets values.|

## Customization

This step requires an Enterprise workspace to gain access to this feature. To learn more about creating an Enterprise workspace or upgrading to an Enterprise workspace from the Azure portal, please visit our [Workspace page.](https://docs.microsoft.com/azure/machine-learning/service/concept-workspace#upgrade). 

Supported customization includes:
1. Column purpose update: Override feature type for the specified column.
2. Transformer parameter update: Update parameters for the specified transformer. Currently supports Imputer and HashOneHotEncoder.
3. Drop columns: Columns to drop from being featurized.
4. Block transformers: Allow/Block transformers to be used on featurization process.

Create FeaturizationConfig object using API calls

In [ ]:
featurization_config = FeaturizationConfig()
featurization_config.blocked_transformers = ['LabelEncoder']
#featurization_config.drop_columns = ['ERP', 'MMIN']
featurization_config.add_column_purpose('MYCT', 'Numeric')
featurization_config.add_column_purpose('VendorName', 'CategoricalHash')
#default strategy mean, add transformer param for for 3 columns
featurization_config.add_transformer_params('Imputer', ['CACH'], {"strategy": "median"})
featurization_config.add_transformer_params('Imputer', ['CHMIN'], {"strategy": "median"})
featurization_config.add_transformer_params('Imputer', ['PRP'], {"strategy": "most_frequent"})
#featurization_config.add_transformer_params('HashOneHotEncoder', [], {"number_of_bits": 3})

In [ ]:
automl_settings = {
    "enable_early_stopping": True, 
    "experiment_timeout_hours" : 0.2,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'normalized_root_mean_squared_error',
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             featurization=featurization_config,
                             training_data = train_data,
                             label_column_name = label,
                             **automl_settings
                            )

Call the `submit` method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.
In this example, we specify `show_output = True` to print currently running iterations to the console.

In [ ]:
remote_run = experiment.submit(automl_config, show_output = False)

In [ ]:
remote_run

Run the following cell to access previous runs. Uncomment the cell below and update the run_id.

In [ ]:
#from azureml.train.automl.run import AutoMLRun
#experiment_name = 'automl-regression-hardware'
#experiment = Experiment(ws, experiment_name)
#remote_run = AutoMLRun(experiment=experiment, run_id='<run_ID_goes_here')
#remote_run

In [ ]:
remote_run.wait_for_completion()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run_customized, fitted_model_customized = remote_run.get_output()

## Transparency

View updated featurization summary

In [ ]:
custom_featurizer = fitted_model_customized.named_steps['datatransformer']

In [ ]:
custom_featurizer.get_featurization_summary()

is_user_friendly=False allows for more detailed summary for transforms being applied

In [ ]:
custom_featurizer.get_featurization_summary(is_user_friendly=False)

In [ ]:
custom_featurizer.get_stats_feature_type_summary()

## Results

#### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show() 

## Explanations
This step requires an Enterprise workspace to gain access to this feature. To learn more about creating an Enterprise workspace or upgrading to an Enterprise workspace from the Azure portal, please visit our [Workspace page.](https://docs.microsoft.com/azure/machine-learning/service/concept-workspace#upgrade). 
This section will walk you through the workflow to compute model explanations for an AutoML model on your remote compute.

### Retrieve any AutoML Model for explanations

Below we select the some AutoML pipeline from our iterations. The `get_output` method returns the a AutoML run and the fitted model for the last invocation. Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [ ]:
automl_run, fitted_model = remote_run.get_output(metric='r2_score')

### Setup model explanation run on the remote compute
The following section provides details on how to setup an AzureML experiment to run model explanations for an AutoML model on your remote compute.

#### Sample script used for computing explanations
View the sample script for computing the model explanations for your AutoML model on remote compute.

In [ ]:
with open('train_explainer.py', 'r') as cefr:
    print(cefr.read())

#### Substitute values in your sample script
The following cell shows how you change the values in the sample script so that you can change the sample script according to your experiment and dataset.

In [ ]:
import shutil
import os

# create script folder
script_folder = './sample_projects/automl-regression-hardware'
if not os.path.exists(script_folder):
    os.makedirs(script_folder)

# Copy the sample script to script folder.
shutil.copy('train_explainer.py', script_folder)

# Create the explainer script that will run on the remote compute.
script_file_name = script_folder + '/train_explainer.py'

# Open the sample script for modification
with open(script_file_name, 'r') as cefr:
    content = cefr.read()

# Replace the values in train_explainer.py file with the appropriate values
content = content.replace('<<experimnet_name>>', automl_run.experiment.name) # your experiment name.
content = content.replace('<<run_id>>', automl_run.id) # Run-id of the AutoML run for which you want to explain the model.
content = content.replace('<<target_column_name>>', 'ERP') # Your target column name
content = content.replace('<<task>>', 'regression') # Training task type
# Name of your training dataset register with your workspace
content = content.replace('<<train_dataset_name>>', 'machineData_train_dataset') 
# Name of your test dataset register with your workspace
content = content.replace('<<test_dataset_name>>', 'machineData_test_dataset')

# Write sample file into your script folder.
with open(script_file_name, 'w') as cefw:
    cefw.write(content)

#### Create conda configuration for model explanations experiment
We need `azureml-explain-model`, `azureml-train-automl` and `azureml-core` packages for computing model explanations for your AutoML model on remote compute.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
azureml_pip_packages = [
    'azureml-train-automl', 'azureml-core', 'azureml-explain-model'
]

# specify CondaDependencies obj
conda_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    pip_packages=azureml_pip_packages)

#### Submit the experiment for model explanations
Submit the experiment with the above `run_config` and the sample script for computing explanations.

In [ ]:
# Now submit a run on AmlCompute for model explanations
from azureml.core.script_run_config import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=script_folder,
                                    script='train_explainer.py',
                                    run_config=conda_run_config)

run = experiment.submit(script_run_config)

# Show run details
run

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

### Feature importance  and  explanation dashboard
In this section we describe how you can download the explanation results from the explanations experiment and visualize the feature importance for your AutoML model. 

#### Setup for visualizing the model explanation results
For visualizing the explanation results for the *fitted_model* we need to perform the following steps:-
1. Featurize test data samples.

The *automl_explainer_setup_obj* contains all the structures from above list. 

In [ ]:
X_test = test_data.drop_columns([label]).to_pandas_dataframe()

In [ ]:
from azureml.train.automl.runtime.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations
explainer_setup_class = automl_setup_model_explanations(fitted_model, 'regression', X_test=X_test)

#### Download engineered feature importance from artifact store
You can use *ExplanationClient* to download the engineered feature explanations from the artifact store of the *automl_run*. You can also use ExplanationDashboard to view the dash board visualization of the feature importance values of the engineered features.

In [ ]:
from azureml.explain.model._internal.explanation_client import ExplanationClient
from interpret_community.widget import ExplanationDashboard
client = ExplanationClient.from_run(automl_run)
engineered_explanations = client.download_model_explanation(raw=False)
print(engineered_explanations.get_feature_importance_dict())
ExplanationDashboard(engineered_explanations, explainer_setup_class.automl_estimator, datasetX=explainer_setup_class.X_test_transform)

#### Download raw feature importance from artifact store
You can use *ExplanationClient* to download the raw feature explanations from the artifact store of the *automl_run*. You can also use ExplanationDashboard to view the dash board visualization of the feature importance values of the raw features.

In [ ]:
raw_explanations = client.download_model_explanation(raw=True)
print(raw_explanations.get_feature_importance_dict())
ExplanationDashboard(raw_explanations, explainer_setup_class.automl_pipeline, datasetX=explainer_setup_class.X_test_raw)

## Operationailze
In this section we will show how you can operationalize an AutoML model and the explainer which was used to compute the explanations in the previous section.

### Register the AutoML model and the scoring explainer
We use the *TreeScoringExplainer* from *azureml.explain.model* package to create the scoring explainer which will be used to compute the raw and engineered feature importances at the inference time. 
In the cell below, we register the AutoML model and the scoring explainer with the Model Management Service.

In [ ]:
# Register trained automl model present in the 'outputs' folder in the artifacts
original_model = automl_run.register_model(model_name='automl_model', 
                                           model_path='outputs/model.pkl')
scoring_explainer_model = automl_run.register_model(model_name='scoring_explainer',
                                                    model_path='outputs/scoring_explainer.pkl')

### Create the conda dependencies for setting up the service
We need to create the conda dependencies comprising of the *azureml-explain-model*, *azureml-train-automl* and *azureml-defaults* packages. 

In [ ]:
myenv = automl_run.get_environment().python.conda_dependencies

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

with open("myenv.yml","r") as f:
    print(f.read())

### View your scoring file

In [ ]:
with open("score_explain.py","r") as f:
    print(f.read())

### Deploy the service
In the cell below, we deploy the service using the conda file and the scoring file from the previous steps. 

In [ ]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Machine Data",  
                                                     "method" : "local_explanation"}, 
                                               description='Get local explanations for Machine test data')

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score_explain.py",
                                   conda_file="myenv.yml")

# Use configs and models generated above
service = Model.deploy(ws, 'model-scoring', [scoring_explainer_model, original_model], inference_config, aciconfig)
service.wait_for_deployment(show_output=True)

### View the service logs

In [ ]:
service.get_logs()

### Inference using some test data
Inference using some test data to see the predicted value from autml model, view the engineered feature importance for the predicted value and raw feature importance for the predicted value.

In [ ]:
if service.state == 'Healthy':
    # Serialize the first row of the test data into json
    X_test_json = X_test[:1].to_json(orient='records')
    print(X_test_json)
    # Call the service to get the predictions and the engineered and raw explanations
    output = service.run(X_test_json)
    # Print the predicted value
    print(output['predictions'])
    # Print the engineered feature importances for the predicted value
    print(output['engineered_local_importance_values'])
    # Print the raw feature importances for the predicted value
    print(output['raw_local_importance_values'])

### Delete the service
Delete the service once you have finished inferencing.

In [ ]:
service.delete()

## Test

In [ ]:
# preview the first 3 rows of the dataset

test_data = test_data.to_pandas_dataframe()
y_test = test_data['ERP'].fillna(0)
test_data = test_data.drop('ERP', 1)
test_data = test_data.fillna(0)


train_data = train_data.to_pandas_dataframe()
y_train = train_data['ERP'].fillna(0)
train_data = train_data.drop('ERP', 1)
train_data = train_data.fillna(0)

In [ ]:
y_pred_train = fitted_model.predict(train_data)
y_residual_train = y_train - y_pred_train

y_pred_test = fitted_model.predict(test_data)
y_residual_test = y_test - y_pred_test

In [ ]:
%matplotlib inline
from sklearn.metrics import mean_squared_error, r2_score

# Set up a multi-plot chart.
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw = {'width_ratios':[1, 1], 'wspace':0, 'hspace': 0})
f.suptitle('Regression Residual Values', fontsize = 18)
f.set_figheight(6)
f.set_figwidth(16)

# Plot residual values of training set.
a0.axis([0, 360, -100, 100])
a0.plot(y_residual_train, 'bo', alpha = 0.5)
a0.plot([-10,360],[0,0], 'r-', lw = 3)
a0.text(16,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_train, y_pred_train))), fontsize = 12)
a0.text(16,140,'R2 score = {0:.2f}'.format(r2_score(y_train, y_pred_train)),fontsize = 12)
a0.set_xlabel('Training samples', fontsize = 12)
a0.set_ylabel('Residual Values', fontsize = 12)

# Plot residual values of test set.
a1.axis([0, 90, -100, 100])
a1.plot(y_residual_test, 'bo', alpha = 0.5)
a1.plot([-10,360],[0,0], 'r-', lw = 3)
a1.text(5,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred_test))), fontsize = 12)
a1.text(5,140,'R2 score = {0:.2f}'.format(r2_score(y_test, y_pred_test)),fontsize = 12)
a1.set_xlabel('Test samples', fontsize = 12)
a1.set_yticklabels([])

plt.show()

In [ ]:
%matplotlib inline
test_pred = plt.scatter(y_test, y_pred_test, color='')
test_test = plt.scatter(y_test, y_test, color='g')
plt.legend((test_pred, test_test), ('prediction', 'truth'), loc='upper left', fontsize=8)
plt.show()